In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.train3d import *

In [2]:
torch.cuda.set_device(3)

In [3]:
df_any.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
ID_66891ac22,CT,ID_42940b2c,ID_17e33f43d0,ID_e14dd0090b,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,0,0,0
ID_8e6e5b51f,CT,ID_76fbed32,ID_1d8eaa14ef,ID_e3919709a0,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,0,0,0,1
ID_cb8b9b514,CT,ID_20039b63,ID_1cfe3e70dd,ID_e3b5d8d9b8,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_ee683911f,CT,ID_a065f3ac,ID_9c727ac231,ID_99e83a310d,,-125.000000,1.0,1,MONOCHROME2,512,...,1.0,40.0,1.0,80.0,0,0,0,0,0,0


In [4]:
dsrc = get_3d_dsrc(df_any)

In [5]:
nrm = Normalize.from_stats(mean,std)
batch_tfms = L(nrm, Cuda(), IntToFloatTensor())

In [29]:
dbunch = DataBunch(
    TfmdDL(dsrc.train, bs=1, after_batch=batch_tfms, num_workers=8, shuffle=True),
    TfmdDL(dsrc.valid, bs=1, after_batch=batch_tfms, num_workers=8)
)
dbunch.device = default_device()
dbunch.c = 6

In [30]:
x,y = dbunch.one_batch()
x.shape, y.shape

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 32 and 36 in dimension 1 at /opt/conda/conda-bld/pytorch_1573049306803/work/aten/src/TH/generic/THTensor.cpp:689

## Model

In [10]:
# #export
# def get_cnn_learner(dbch, model, lf, splitter, pretrained=False, opt_func=None, metrics=None, fp16=True, config=None):
#     if metrics is None: metrics=[accuracy_multi,accuracy_any]
#     if opt_func is None: opt_func = partial(Adam, wd=1e-5, eps=1e-4, sqr_mom=0.999)
#     if config is None: config=dict(ps=0., lin_ftrs=[], concat_pool=False)
#     learn = Learner(dbunch, model, loss_func=lf, splitter=splitter, **kwargs)
# #     learn = cnn_learner(dbch, arch, pretrained=pretrained, loss_func=lf, lr=3e-3,
# #                         opt_func=opt_func, metrics=metrics, config=config)
#     return learn.to_fp16() if fp16 else learn

In [11]:
metrics = [accuracy_multi,accuracy_any]
opt_func = partial(Adam, wd=1e-5, eps=1e-4, sqr_mom=0.999)

In [16]:
from fastai2.vision.learner import _xresnet_split

In [25]:
arch = xresnet34
split = _xresnet_split
cut = -3
m_body = create_body(arch, pretrained=False, cut=cut)

In [12]:
head = nn.Sequential(Batchify(),
    conv3(512,256,2), # 8
    conv3(256,128,2), # 4
    conv3(128, 64,2), # 2
    DeBatchify(), nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(64,6))

init_cnn(m)

In [ ]:
#export
class Rehsape(Module):
    def forward(self, x): return x
class Batchify(Module):
    def forward(self, x): return x[None].transpose(1,2)

class DeBatchify(Module):
    def forward(self, x): return x[0].transpose(0,1)

def conv3(ni,nf,stride=1):
    return ConvLayer(ni, nf, (5,3,3), stride=(1,stride,stride), ndim=3, padding=(2,1,1))

In [ ]:
class Model3D(nn.Module):
    def __init__(self, body):
        self.body = body
        self.body3d = nn.Sequential(
            conv3(512,256,2), # 8
            conv3(256,128,2), # 4
            conv3(128, 64,2), # 2
        )
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten(), nn.Linear(64,6))

    def forward(self, x):
        shape = x.shape
        print('X', shape)
        x = x.reshape(-1, **shape[2:])
        print('X.reshape', x.shape)
        x = self.body(x)
        print('body', x.shape)
        x = x.transpose(1,2)
        print('body.t', x.shape)
        x = 
        
        
        self.body()

In [26]:
m_body

Sequential(
  (0): ConvLayer(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (1): ConvLayer(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (2): ConvLayer(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): ResBlock(
      (convs): Sequential(
        (0): ConvLayer(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_

In [21]:
model

Sequential(
  (0): Sequential(
    (0): ConvLayer(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): ConvLayer(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): ConvLayer(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): ResBlock(
        (convs): Sequential(
          (0): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): Batch

In [ ]:
m = resnet34()

In [13]:
config=dict(custom_head=m, init=None)
learn = get_learner(dbunch, xresnet34, get_loss(), config=config)

In [14]:
learn.summary()

Sequential (Input shape: 28 x 3 x 256 x 256)
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               28 x 32 x 128 x 128  864        True      
________________________________________________________________
BatchNorm2d          28 x 32 x 128 x 128  64         True      
________________________________________________________________
ReLU                 28 x 32 x 128 x 128  0          False     
________________________________________________________________
Conv2d               28 x 32 x 128 x 128  9,216      True      
________________________________________________________________
BatchNorm2d          28 x 32 x 128 x 128  64         True      
________________________________________________________________
ReLU                 28 x 32 x 128 x 128  0          False     
________________________________________________________________
Conv2d               28 x 64 x 128 x 128  18,432     True      
_____________________________________________________

## Training

In [18]:
name = 'train3d_baseline'

In [19]:
do_fit(learn, 1, 4e-2, freeze=False)
learn.save(f'runs/{name}-1')

KeyboardInterrupt: 

In [ ]:
learn.dbunch = get_data(512, 224)
do_fit(learn, 12, 5e-3, freeze=False)
learn.save(f'runs/{name}-2')

## Submission

In [19]:
learn.load(f'runs/{name}-3')
sub_fn = f'subm/{name}'

In [ ]:
learn.dbunch = get_test_data(df_tst, bs=1024, sz=224)
preds,targs = learn.get_preds()

In [33]:
pred_csv = submission(df_tst, preds, fn=nm)
FileLink(f'{sub_fn}.csv')

In [36]:
api.competition_submit(f'{nm}.csv', '0.0744,rn34_repl (scl 0.88)', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.52MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [ ]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

## Export

In [3]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data_01_metadata_stage2.ipynb.
Converted 01_data_02_preprocess_windows.ipynb.
Converted 02_train_01_train.ipynb.
Converted 04_orig_replace_ashaw_refactor.ipynb.
Converted 04_replace_ashaw_refactor.ipynb.
Converted 04b_orig_replace_ashaw_refactor.ipynb.
Converted 10_qure.ipynb.
Converted 12_merge.ipynb.
Converted 14_xgboost.ipynb.
Converted 16_slice_e2e-shallow.ipynb.
Converted 16b_orig_slice_e2e-shallow.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 17_slice_model-deep.ipynb.
Converted 21_cleanup-nocrop2.ipynb.
Converted 26_submit_final.ipynb.
Converted 27_ensemble_tabular_nn.ipynb.
Converted 99_index.ipynb.
Converted cleanup-combine-qure.ipynb.
Converted delete_03b_cleanup-tif.ipynb.
Converted submit.ipynb.
Converted walkthru.ipynb.
Converted x00_tcia-ct-segm-prep.ipynb.
Converted x00_tcia-ct-segm-train.ipynb.
